In [6]:
import xarray as xr
import pandas as pd
import numpy as np
import tools.config as config
import os
from tools.helper_plot import set_plot_style, draw_12_price, draw_10_price, draw_22_price
import os
import pandas as pd
from collections import namedtuple
import tools.config as config
from tools.helper_plot import set_plot_style,xarray_to_dict, get_global_ylim, plot_13_layout,get_colors, plot_25_layout
from tools.tools import (
    get_path, get_year, save2nc, filter_all_from_dims, nc_to_tif, get_data_RES
)


task_name = config.TASK_NAME
input_dir = f'../../../output/{task_name }/carbon_price/1_draw_data'
output_dir = f"../../../output/{task_name }/carbon_price/3_Paper_figure"
input_file = "xr_total_carbon"
total_name = 'Total'

In [7]:
data_dict = xarray_to_dict(f"{input_dir}/{input_file}_original.nc",1,total_name=total_name)

In [21]:
data_dict['Run_25_GHG_off_BIO_off_CUT_10']
filename = 'Run_01_GHG_high_BIO_high_CUT_50'

In [22]:
data = get_data_RES(config.TASK_NAME, filename)

In [23]:
dir(data)

['AC_COST_MULTS',
 'AF_EST_COST_HA',
 'AG2EP_TRANSITION_COSTS_HA',
 'AGEC_CROPS',
 'AGEC_LVSTK',
 'AGGHG_CROPS',
 'AGGHG_IRRPAST',
 'AGGHG_LVSTK',
 'AGLU2DESC',
 'AGRICULTURAL_LANDUSES',
 'AGTECH_EI_DATA',
 'AG_L_MRJ',
 'AG_MAN_DESC',
 'AG_MAN_LU_DESC',
 'AG_MAN_L_MRJ_DICT',
 'AG_MAN_MAP',
 'AG_TMATRIX',
 'AG_TO_DESTOCKED_NATURAL_COSTS_HA',
 'ALLDESC2LU',
 'ALLLU2DESC',
 'ALL_LANDUSES',
 'ASPARAGOPSIS_DATA',
 'BASE_YR_GBF2_score',
 'BASE_YR_GHG_t',
 'BASE_YR_economic_value',
 'BASE_YR_overall_bio_value',
 'BASE_YR_production_t',
 'BASE_YR_water_ML',
 'BAU_PROD_INCR',
 'BAU_PROD_INCR_xr',
 'BECCS_COSTS_AUD_HA_YR',
 'BECCS_COST_MULTS',
 'BECCS_MWH_HA_YR',
 'BECCS_REV_AUD_HA_YR',
 'BECCS_REV_MULTS',
 'BECCS_TCO2E_HA_YR',
 'BIOCHAR_DATA',
 'BIO_GBF2_BASE_YR',
 'BIO_GBF2_MASK',
 'BIO_GBF2_MASK_LDS',
 'BIO_HABITAT_CONTRIBUTION_LOOK_UP',
 'BIO_QUALITY_LDS',
 'BIO_QUALITY_RAW',
 'CARBON_PRICES',
 'CLIMATE_CHANGE_IMPACT',
 'CLIMATE_CHANGE_IMPACT_xr',
 'CM2LU_IDX',
 'CM2PR_DICT',
 'CM_CROPS',
 '

In [24]:





def get_GBF2_target_list_start_to_2050(
    data,
    task_name: str,
    filename: str,
    start_year: int,
    end_year: int = 2050,
) -> list[float]:
    """
    返回从 start_year 到 2050（含）的 GBF2 target 列表。

    Returns
    -------
    list[float]
        长度 = end_year - start_year + 1，对应每一年的 target（绝对值，不是百分比）
    """
    import os
    import importlib.util
    import numpy as np
    from scipy.interpolate import interp1d

    def _import_settings_from_output(task_name: str, filename: str):
        """
        从输出目录按路径导入 settings.py
        期望路径: ../../../output/{task_name}/{filename}/luto/settings.py
        """
        settings_path = os.path.abspath(
            os.path.join("../../../output", task_name, filename, "luto", "settings.py")
        )
        if not os.path.isfile(settings_path):
            raise FileNotFoundError(f"settings.py 不存在: {settings_path}")

        spec = importlib.util.spec_from_file_location("custom_luto_settings", settings_path)
        if spec is None or spec.loader is None:
            raise ImportError(f"无法为 settings.py 创建 spec: {settings_path}")

        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module

    settings = _import_settings_from_output(task_name, filename)

    target_config = settings.GBF2_TARGETS_DICT.get(settings.BIODIVERSITY_TARGET_GBF_2)
    if target_config is None:
        target_config = {2030: 0, 2050: 0, 2100: 0}

    bio_habitat_score_baseline_sum = (data.BIO_GBF2_MASK * data.REAL_AREA).sum()
    bio_habitat_score_base_yr_sum = data.BIO_GBF2_BASE_YR.sum()
    bio_habitat_score_base_yr_proportion = bio_habitat_score_base_yr_sum / bio_habitat_score_baseline_sum

    bio_habitat_target_proportion = [
        bio_habitat_score_base_yr_proportion + ((1 - bio_habitat_score_base_yr_proportion) * i)
        for i in target_config.values()
    ]

    targets_key_years = {
        data.YR_CAL_BASE: bio_habitat_score_base_yr_sum,
        **dict(zip(
            target_config.keys(),
            bio_habitat_score_baseline_sum * np.array(bio_habitat_target_proportion)
        ))
    }

    f = interp1d(
        list(targets_key_years.keys()),
        list(targets_key_years.values()),
        kind="linear",
        fill_value="extrapolate"
    )

    years = np.arange(int(start_year), int(end_year) + 1, dtype=int)
    targets = f(years).astype(float).tolist()
    return targets

In [25]:
data = get_data_RES(config.TASK_NAME, filename)
targets_2025_2050 = get_GBF2_target_list_start_to_2050(
    data=data,
    task_name=config.TASK_NAME,
    filename=filename,
    start_year=config.START_YEAR,
    end_year=2050,
)

In [26]:
targets_2025_2050

[179214755.4051783,
 181107042.9263684,
 182999330.4475585,
 184891617.9687486,
 186783905.48993868,
 188676193.01112878,
 189937718.0252555,
 191199243.03938225,
 192460768.05350897,
 193722293.06763572,
 194983818.08176243,
 196245343.09588915,
 197506868.1100159,
 198768393.12414262,
 200029918.13826936,
 201291443.15239608,
 202552968.1665228,
 203814493.18064955,
 205076018.19477627,
 206337543.208903,
 207599068.22302973,
 208860593.23715645,
 210122118.2512832,
 211383643.26540992,
 212645168.27953666,
 213906693.29366338]